In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.

import os
print(os.listdir("../input"))

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
is_correct_version = '1.13.' in tf.__version__
assert is_correct_version , "Wrong Tensorflow Version [{}]".format(tf.__version__)

is_eager_enabled = tf.executing_eagerly()
assert is_eager_enabled, "Eager mode not activated"

In [ ]:
train_labels = pd.read_csv("../input/csvTrainLabel 13440x1.csv")
test_labels = pd.read_csv("../input/csvTestLabel 3360x1.csv")
train_images = pd.read_csv("../input/csvTrainImages 13440x1024.csv")
test_images = pd.read_csv("../input/csvTestImages 3360x1024.csv")
print(train_images.head())


In [ ]:
train_images = np.expand_dims(train_images, axis=-1)
train_labels = np.int64(train_labels)
test_images = np.expand_dims(test_images, axis=-1)
test_labels = np.int64(test_labels)

In [ ]:
formatted_train_images = np.ndarray(shape=(len(train_images), 32, 32, 1),dtype=float)
formatted_test_images = np.ndarray(shape=(len(test_images), 32, 32, 1), dtype=float)

for i in range(len(train_images)):
    image = np.transpose(np.reshape(train_images[i]/255., (1, 32,32)))
    formatted_train_images[i] = np.array(image)
    
for i in range(len(test_images)):
    image = np.transpose(np.reshape(test_images[i]/255., (1, 32,32)))
    formatted_test_images[i] = np.array(image)  


In [ ]:
plt.figure(figsize=(10, 10))
random_idx = np.random.choice(len(train_images), 36)
for i in range(36):
    plt.subplot(6, 6, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_idx[i]
    image = np.squeeze(formatted_train_images[image_ind])
    plt.imshow(image)
    plt.xlabel(train_labels.flatten()[image_ind])

In [ ]:
def build_cnn_model():
  cnn_model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3), input_shape=(32, 32, 1), activation=tf.nn.relu),       
      tf.keras.layers.MaxPool2D(pool_size=(2,2)), 
      tf.keras.layers.Conv2D(filters=48, kernel_size=(3,3), input_shape=(16, 16, 1), activation=tf.nn.relu),
      tf.keras.layers.MaxPool2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      tf.keras.layers.Dense(28, activation=tf.nn.softmax)
  ])
  return cnn_model

model = build_cnn_model()

In [ ]:
model.compile(optimizer=tf.train.GradientDescentOptimizer(learning_rate=1e-1), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 10
EPOCHS = 4

In [ ]:

print(formatted_train_images.shape)

In [ ]:
model.fit(formatted_train_images, train_labels.flatten(), batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
test_loss, test_acc = model.evaluate(formatted_test_images, test_labels)

print('Test accuracy:', test_acc)